<a href="https://colab.research.google.com/github/bgsw404notfound/SkiSphe/blob/main/Vis%26Dash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import plotly.express as px
from ipywidgets import VBox, HBox, Checkbox, Output, Button, Layout, Label, RadioButtons
from IPython.display import display, clear_output, HTML
import urllib.parse
import plotly.io as pio
pio.renderers.default = 'notebook'  # Alternatives: 'iframe', 'svg', 'browser'

# Load dataset
df = pd.read_csv('training_assignments_output.csv')

# Fill NaNs
df[['L1', 'L2', 'L3', 'L4']] = df[['L1', 'L2', 'L3', 'L4']].fillna('')
df[['skills_recommended_by_manager', 'skills_recommended_by_department',
    'skills_selected_for_own', 'training_details']] = df[[
    'skills_recommended_by_manager', 'skills_recommended_by_department',
    'skills_selected_for_own', 'training_details'
]].fillna('')

# Create skill-to-level mapping
def extract_skills_with_levels(row):
    level_skill_map = {}
    for level in ['L1', 'L2', 'L3', 'L4']:
        skills = [s.strip() for s in row[level].split(',') if s.strip()]
        for skill in skills:
            level_skill_map[skill] = level
    return level_skill_map

df['Skill_Level_Map'] = df.apply(extract_skills_with_levels, axis=1)
df['All_Skills_With_Level'] = df['Skill_Level_Map'].apply(lambda x: list(x.items()))
df['Skill_List'] = df['Skill_Level_Map'].apply(lambda x: list(x.keys()))

# Unique filters
associate_names = sorted(df['name'].dropna().unique())
roles = sorted(df['role'].dropna().unique())
all_skills = sorted(set(skill for skills in df['Skill_List'] for skill in skills))

# UI Helpers
def build_checkbox_section(options, label_text):
    checkboxes = [Checkbox(value=False, description=opt, indent=False) for opt in options]
    box = VBox(checkboxes, layout=Layout(height='200px', overflow='auto', border='1px solid #ccc'))

    select_all_btn = Button(description="Select All", layout=Layout(width='90px'))
    clear_all_btn = Button(description="Clear All", layout=Layout(width='90px'))

    def select_all(_):
        for cb in checkboxes:
            cb.value = True

    def clear_all(_):
        for cb in checkboxes:
            cb.value = False

    select_all_btn.on_click(select_all)
    clear_all_btn.on_click(clear_all)

    header = HBox([Label(value=label_text), select_all_btn, clear_all_btn])
    return VBox([header, box]), checkboxes

# Role selection
role_selector = RadioButtons(
    options=['Associate', 'Manager'],
    description='Select Role:',
    disabled=False
)

# Filter widgets (hidden initially)
name_section, name_checkboxes = build_checkbox_section(associate_names, "Names")
role_section, role_checkboxes = build_checkbox_section(roles, "Roles")
skill_section, skill_checkboxes = build_checkbox_section(all_skills, "Skills")

# Output area
output_area = Output()
filter_button = Button(description="Apply Filters", button_style='success')

# Show/hide filters based on role selection
def update_filters(change):
    with output_area:
        clear_output()
        if change['new'] == 'Associate':
            display(name_section)
            display(filter_button)
        else:
            display(name_section)
            display(role_section)
            display(skill_section)
            display(filter_button)

role_selector.observe(update_filters, names='value')

# Custom display function to avoid JupyterLab version check issues
def safe_display(fig):
    try:
        fig.show()
    except Exception as e:
        print(f"Error displaying figure: {str(e)}")
        display(fig)

# Filter function
def on_filter_click(_):
    with output_area:
        clear_output()
        filtered_df = df.copy()
        selected_role = role_selector.value

        if selected_role == 'Associate':
            selected_names = [cb.description for cb in name_checkboxes if cb.value]
            if selected_names:
                filtered_df = filtered_df[filtered_df['name'].isin(selected_names)]
        else:
            selected_roles = [cb.description for cb in role_checkboxes if cb.value]
            selected_skills = [cb.description for cb in skill_checkboxes if cb.value]
            if selected_roles:
                filtered_df = filtered_df[filtered_df['role'].isin(selected_roles)]
            if selected_skills:
                filtered_df = filtered_df[filtered_df['Skill_List'].apply(
                    lambda x: all(skill in x for skill in selected_skills)
                )]

        display_cols = ['name', 'role', 'L1', 'L2', 'L3', 'L4',
                       'skills_recommended_by_manager',
                       'skills_recommended_by_department',
                       'skills_selected_for_own',
                       'training_details',
                       'available_learning_time',
                       'learning_time_utilized_previous_year']
        display(filtered_df[display_cols])

        # Convert time strings to numeric values
        def parse_time(time_str):
            if pd.isna(time_str) or time_str == '':
                return 0
            try:
                return float(time_str.split()[0])
            except:
                return 0

        filtered_df['available_learning_time_num'] = filtered_df['available_learning_time'].apply(parse_time)
        filtered_df['learning_time_utilized_previous_year_num'] = filtered_df['learning_time_utilized_previous_year'].apply(parse_time)

        # Learning Time Utilization Bar Chart (for both roles)
        time_comparison = filtered_df[['available_learning_time_num', 'learning_time_utilized_previous_year_num']].mean().reset_index()
        time_comparison.columns = ['Time Type', 'Hours']

        fig_time_comparison = px.bar(
            time_comparison,
            x='Time Type',
            y='Hours',
            color='Time Type',
            title='Average Available vs Utilized Learning Time (Hours)',
            text='Hours'
        )
        fig_time_comparison.update_traces(texttemplate='%{text:.1f}', textposition='outside')
        safe_display(fig_time_comparison)

        if selected_role == 'Associate':
            # Associate-specific visualizations

            # Top Skills by Level Chart
            skill_level_data = []
            for item in filtered_df['All_Skills_With_Level']:
                skill_level_data.extend(item)
            skill_level_df = pd.DataFrame(skill_level_data, columns=['Skill', 'Level'])

            if not skill_level_df.empty:
                # Top Skills by Level (Horizontal)
                top_skills_combined = skill_level_df.groupby(['Skill', 'Level']).size().reset_index(name='Count')
                top_skills_combined = top_skills_combined.sort_values('Count', ascending=False).groupby('Level').head(10)

                fig_top_combined = px.bar(
                    top_skills_combined,
                    x='Count',
                    y='Skill',
                    color='Level',
                    barmode='group',
                    orientation='h',
                    title='Top Skills by Level',
                    category_orders={'Level': ['L1', 'L2', 'L3', 'L4']}
                )
                fig_top_combined.update_layout(yaxis={'categoryorder': 'total ascending'})
                safe_display(fig_top_combined)

                # Skill Heatmap by Level
                heatmap_df = skill_level_df.groupby(['Skill', 'Level']).size().unstack(fill_value=0).reindex(columns=['L1', 'L2', 'L3', 'L4'], fill_value=0)
                fig_heatmap = px.imshow(heatmap_df, text_auto=True, aspect="auto",
                                      title="Skill Heatmap by Level",
                                      labels=dict(x="Skill Level", y="Skill", color="Count"))
                safe_display(fig_heatmap)

            # Training Recommendations by Source (Sunburst)
            sunburst_data = []
            for col, source in [
                ('skills_recommended_by_manager', 'Manager'),
                ('skills_recommended_by_department', 'Department'),
                ('skills_selected_for_own', 'Self')
            ]:
                for i, row in filtered_df.iterrows():
                    entries = row[col]
                    for skill in entries.split(','):
                        if '-' in skill:
                            skill_name = skill.strip().split('-')[0].strip()
                            sunburst_data.append((source, skill_name))

            if sunburst_data:
                sb_df = pd.DataFrame(sunburst_data, columns=['Source', 'Skill'])
                fig4 = px.sunburst(sb_df, path=['Source', 'Skill'], title='Training Recommendations by Source')
                safe_display(fig4)

            # Certifications Summary
            cert_df = filtered_df[['certifications_obtained', 'certifications_in_progress']].fillna('')
            cert_df['certifications_obtained'] = cert_df['certifications_obtained'].apply(lambda x: len(x.split(',')) if x else 0)
            cert_df['certifications_in_progress'] = cert_df['certifications_in_progress'].apply(lambda x: len(x.split(',')) if x else 0)

            cert_summary = cert_df.mean().reset_index()
            cert_summary.columns = ['Certification Type', 'Average Count']

            fig_cert = px.bar(cert_summary, x='Certification Type', y='Average Count',
                              title='Average Certifications Obtained vs In Progress',
                              color='Certification Type', text='Average Count')
            fig_cert.update_traces(texttemplate='%{text:.1f}', textposition='outside')
            safe_display(fig_cert)

            # Training Cards (Interactive)
            training_cards = []
            for _, row in filtered_df.iterrows():
                name = row['name']
                role = row['role']
                trainings = [t.strip() for t in row['training_details'].split(',') if t.strip()]

                if not trainings:
                    continue

                training_links = ''.join(
                    f'<li><a href="about:blank" onclick="window.open(\'data:text/html,{urllib.parse.quote(f"<h1>{t}</h1>")}\', \'_blank\'); return false;">{t}</a></li>'
                    for t in trainings
                )

                card_html = f"""
                <div style="
                    border: 1px solid #ccc;
                    border-radius: 10px;
                    padding: 10px;
                    margin: 10px;
                    width: 300px;
                    display: inline-block;
                    vertical-align: top;
                    background-color: #f9f9f9;
                ">
                    <h4 style="margin: 0 0 5px 0;">{name}</h4>
                    <p style="margin: 0 0 8px 0; color: #555;"><strong>Role:</strong> {role}</p>
                    <p style="margin-bottom: 5px;"><strong>Recommended Trainings:</strong></p>
                    <ul style="padding-left: 20px; margin: 0;">
                        {training_links}
                    </ul>
                </div>
                """
                training_cards.append(card_html)

            display(HTML(''.join(training_cards)))

        else:
            # Manager-specific visualizations

            # Skill Level Distribution Chart
            skill_level_data = []
            for item in filtered_df['All_Skills_With_Level']:
                skill_level_data.extend(item)
            skill_level_df = pd.DataFrame(skill_level_data, columns=['Skill', 'Level'])

            if not skill_level_df.empty:
                level_counts = skill_level_df['Level'].value_counts().sort_index().reset_index()
                level_counts.columns = ['Skill Level', 'Count']
                fig1 = px.bar(level_counts, x='Skill Level', y='Count',
                              title='Number of Skills by Level')
                safe_display(fig1)

                # Skill Heatmap by Level
                heatmap_df = skill_level_df.groupby(['Skill', 'Level']).size().unstack(fill_value=0).reindex(columns=['L1', 'L2', 'L3', 'L4'], fill_value=0)
                fig_heatmap = px.imshow(heatmap_df, text_auto=True, aspect="auto",
                                      title="Skill Heatmap by Level",
                                      labels=dict(x="Skill Level", y="Skill", color="Count"))
                safe_display(fig_heatmap)

            # Certifications Summary
            cert_df = filtered_df[['certifications_obtained', 'certifications_in_progress']].fillna('')
            cert_df['certifications_obtained'] = cert_df['certifications_obtained'].apply(lambda x: len(x.split(',')) if x else 0)
            cert_df['certifications_in_progress'] = cert_df['certifications_in_progress'].apply(lambda x: len(x.split(',')) if x else 0)

            cert_summary = cert_df.mean().reset_index()
            cert_summary.columns = ['Certification Type', 'Average Count']

            fig_cert = px.bar(cert_summary, x='Certification Type', y='Average Count',
                              title='Average Certifications Obtained vs In Progress',
                              color='Certification Type', text='Average Count')
            fig_cert.update_traces(texttemplate='%{text:.1f}', textposition='outside')
            safe_display(fig_cert)


        # ---- Grade-Based Analysis (All Data) ----
        grade_df = df.groupby('grade')['trainings_to_assign'].count().reset_index()
        grade_df.columns = ['Grade', 'Assigned Trainings Count']
        fig_grade = px.bar(
            grade_df,
            x='Grade',
            y='Assigned Trainings Count',
            title='Training Assignments by Grade (All Associates)',
            text='Assigned Trainings Count',
            color='Grade'
        )
        fig_grade.update_traces(textposition='outside')
        fig_grade.show()

        # ---- Role-Based Analysis (All Data) ----
        role_df = df.groupby('role')['trainings_to_assign'].count().reset_index()
        role_df.columns = ['Role', 'Assigned Trainings Count']
        fig_role = px.bar(
            role_df,
            x='Role',
            y='Assigned Trainings Count',
            title='Training Assignments by Role (All Associates)',
            text='Assigned Trainings Count',
            color='Role'
        )
        fig_role.update_traces(textposition='outside')
        fig_role.update_layout(xaxis_tickangle=-45)
        fig_role.show()


# Attach handler
filter_button.on_click(on_filter_click)

# Initial layout
display(VBox([
    role_selector,
    output_area
]))